In [5]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)

In [12]:
data <- read_csv("sonar.all-data.csv", col_names=FALSE)
data

information <- read_csv("sonar.names.csv")
information

Parsed with column specification:
cols(
  .default = col_double(),
  X61 = col_character()
)

See spec(...) for full column specifications.



X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,⋯,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,⋯,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,⋯,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,⋯,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,⋯,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,⋯,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,⋯,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R
0.0317,0.0956,0.1321,0.1408,0.1674,0.1710,0.0731,0.1401,0.2083,0.3513,⋯,0.0201,0.0248,0.0131,0.0070,0.0138,0.0092,0.0143,0.0036,0.0103,R
0.0519,0.0548,0.0842,0.0319,0.1158,0.0922,0.1027,0.0613,0.1465,0.2838,⋯,0.0081,0.0120,0.0045,0.0121,0.0097,0.0085,0.0047,0.0048,0.0053,R
0.0223,0.0375,0.0484,0.0475,0.0647,0.0591,0.0753,0.0098,0.0684,0.1487,⋯,0.0145,0.0128,0.0145,0.0058,0.0049,0.0065,0.0093,0.0059,0.0022,R


Parsed with column specification:
cols(
  `NAME: Sonar` = col_character(),
  `Mines vs. Rocks` = col_character()
)

Warning message:
“86 parsing failures.
row col  expected    actual              file
  1  -- 2 columns 1 columns 'sonar.names.csv'
  2  -- 2 columns 1 columns 'sonar.names.csv'
  3  -- 2 columns 1 columns 'sonar.names.csv'
  4  -- 2 columns 1 columns 'sonar.names.csv'
  5  -- 2 columns 1 columns 'sonar.names.csv'
... ... ......... ......... .................
See problems(...) for more details.
”


NAME: Sonar,Mines vs. Rocks
<chr>,<chr>
SUMMARY: This is the data set used by Gorman and Sejnowski in their study,NA
of the classification of sonar signals using a neural network [1]. The,NA
task is to train a network to discriminate between sonar signals bounced,NA
off a metal cylinder and those bounced off a roughly cylindrical rock.,NA
SOURCE: The data set was contributed to the benchmark collection by Terry,NA
Sejnowski,now at the Salk Institute and the University of California at
San Deigo. The data set was developed in collaboration with R. Paul,NA
Gorman of Allied-Signal Aerospace Technology Center.,NA
MAINTAINER: Scott E. Fahlman,NA
